# Chapter 16: Type Annotation Fundamentals

This notebook covers the foundations of Python's type annotation system. Type hints
make code more readable, enable static analysis with tools like mypy, and improve
IDE autocompletion - all without affecting runtime behavior.

## Key Concepts
- Variable, function parameter, and return type annotations
- Inspecting annotations at runtime with `get_type_hints()`
- Built-in generic types: `list[int]`, `dict[str, int]`, `tuple`, `set`
- `Optional`, `Union`, and the `X | Y` union syntax (Python 3.10+)
- Class annotations and `__init__` type hints
- `None` return type for side-effect functions
- Type aliases with `TypeAlias`

## Variable and Function Annotations

Python 3.0 introduced function annotations (PEP 3107) and Python 3.6 added variable
annotations (PEP 526). Annotations are **not enforced at runtime** - they are metadata
consumed by type checkers, IDEs, and documentation tools.

In [ ]:
# Variable annotations
name: str = "Alice"
age: int = 30
balance: float = 1042.50
is_active: bool = True

# You can annotate without assigning (declaration only)
email: str  # Not yet assigned - no runtime value


# Function annotations: parameters and return type
def greet(name: str, excited: bool = False) -> str:
    """Return a greeting for the given name."""
    base = f"Hello, {name}!"
    return base.upper() if excited else base


print(greet("Alice"))
print(greet("Bob", excited=True))

# Annotations don't prevent wrong types at runtime (no enforcement)
print(greet(42))  # Works at runtime, but mypy would flag this

## Inspecting Annotations at Runtime

Function annotations are stored in `__annotations__`. The `typing.get_type_hints()`
function is the preferred way to access them - it resolves forward references and
string annotations that `__annotations__` leaves as raw strings.

In [ ]:
from typing import get_type_hints


def calculate_bmi(weight_kg: float, height_m: float) -> float:
    """Calculate Body Mass Index."""
    return weight_kg / (height_m ** 2)


# Raw __annotations__ dict
print("__annotations__:", calculate_bmi.__annotations__)

# get_type_hints() is preferred - resolves forward references
hints = get_type_hints(calculate_bmi)
print("get_type_hints():", hints)

# Iterate over parameter types
for param, type_hint in hints.items():
    print(f"  {param}: {type_hint.__name__ if hasattr(type_hint, '__name__') else type_hint}")

# Module-level variable annotations are in the module's __annotations__
print("\nModule-level annotations (sample):")
for var_name, var_type in list(__annotations__.items())[:4]:
    print(f"  {var_name}: {var_type}")

## Built-in Generic Types

Since Python 3.9 (PEP 585), you can use built-in collection types directly as
generics: `list[int]`, `dict[str, float]`, `tuple[str, ...]`, `set[int]`.
Before 3.9, you needed `typing.List`, `typing.Dict`, etc.

In [ ]:
# list[T] - homogeneous sequence
scores: list[int] = [95, 87, 92, 78]

# dict[K, V] - key-value mapping
user_ages: dict[str, int] = {"Alice": 30, "Bob": 25, "Carol": 28}

# set[T] - unique elements
unique_tags: set[str] = {"python", "typing", "tutorial"}

# tuple[T1, T2, ...] - fixed-length, heterogeneous
point: tuple[float, float] = (3.14, 2.72)
record: tuple[str, int, bool] = ("Alice", 30, True)

# tuple[T, ...] - variable-length, homogeneous (the ellipsis is key)
readings: tuple[float, ...] = (98.6, 99.1, 97.8, 98.2)


def average(values: list[float]) -> float:
    """Calculate the average of a list of floats."""
    return sum(values) / len(values)


def merge_dicts(a: dict[str, int], b: dict[str, int]) -> dict[str, int]:
    """Merge two dictionaries, summing values for shared keys."""
    result = dict(a)
    for key, value in b.items():
        result[key] = result.get(key, 0) + value
    return result


print(f"Average: {average([1.0, 2.0, 3.0, 4.0])}")
print(f"Merged: {merge_dicts({'a': 1, 'b': 2}, {'b': 3, 'c': 4})}")


# Nested generics work naturally
matrix: list[list[int]] = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
]

user_data: dict[str, list[str]] = {
    "alice": ["admin", "editor"],
    "bob": ["viewer"],
}

print(f"Matrix: {matrix}")
print(f"User roles: {user_data}")

## Optional and Union Types

- `Union[X, Y]` means the value can be type `X` or type `Y`
- `Optional[X]` is shorthand for `Union[X, None]` - value can be `X` or `None`
- Python 3.10+ introduced `X | Y` syntax as a cleaner alternative to `Union`

In [ ]:
from typing import Optional, Union


# Union: accepts multiple types
def format_id(id_value: Union[int, str]) -> str:
    """Format an ID that might be numeric or string."""
    if isinstance(id_value, int):
        return f"ID-{id_value:06d}"
    return f"ID-{id_value}"


print(format_id(42))
print(format_id("ABC123"))


# Optional: value or None
def find_user(user_id: int) -> Optional[str]:
    """Look up a user by ID; returns None if not found."""
    users = {1: "Alice", 2: "Bob", 3: "Carol"}
    return users.get(user_id)


result = find_user(1)
print(f"Found: {result}")

result = find_user(99)
print(f"Not found: {result}")


# Python 3.10+ union syntax: X | Y (cleaner and preferred)
def format_id_modern(id_value: int | str) -> str:
    """Same as format_id but using the modern | syntax."""
    if isinstance(id_value, int):
        return f"ID-{id_value:06d}"
    return f"ID-{id_value}"


def find_user_modern(user_id: int) -> str | None:
    """Same as find_user using modern syntax. str | None == Optional[str]."""
    users = {1: "Alice", 2: "Bob", 3: "Carol"}
    return users.get(user_id)


print(f"\nModern syntax: {format_id_modern(42)}")
print(f"Modern optional: {find_user_modern(2)}")

## Class Annotations and `__init__` Type Hints

Class-level annotations describe instance attributes. By convention, type hints
for instance attributes go in `__init__`, while class-level annotations describe
the expected shape of the class.

In [ ]:
from typing import get_type_hints


class Product:
    """A product with typed attributes."""

    # Class-level annotations declare the attribute types
    name: str
    price: float
    tags: list[str]
    description: str | None

    def __init__(
        self,
        name: str,
        price: float,
        tags: list[str] | None = None,
        description: str | None = None,
    ) -> None:
        self.name = name
        self.price = price
        self.tags = tags or []
        self.description = description

    def display_price(self) -> str:
        return f"${self.price:.2f}"

    def __repr__(self) -> str:
        return f"Product({self.name!r}, {self.display_price()}, tags={self.tags})"


laptop = Product("Laptop", 999.99, ["electronics", "computers"])
book = Product("Python Cookbook", 49.99, description="Recipes for mastering Python")

print(laptop)
print(book)

# Inspect class annotations
print("\nClass annotations:")
for attr, hint in get_type_hints(Product).items():
    print(f"  {attr}: {hint}")

# Inspect __init__ annotations
print("\n__init__ annotations:")
for param, hint in get_type_hints(Product.__init__).items():
    if param != "return":
        print(f"  {param}: {hint}")

## None Return Type for Side-Effect Functions

Functions that perform side effects (printing, writing files, mutating state) and
don't return a meaningful value should be annotated with `-> None`. This makes the
intent explicit and helps type checkers catch accidental use of the return value.

In [ ]:
class Logger:
    """A simple logger demonstrating None return types."""

    def __init__(self) -> None:
        self._messages: list[str] = []

    def log(self, message: str) -> None:
        """Append a message. Returns nothing (side effect only)."""
        self._messages.append(message)

    def clear(self) -> None:
        """Clear all messages. Returns nothing."""
        self._messages.clear()

    def get_messages(self) -> list[str]:
        """Return a copy of all messages."""
        return list(self._messages)


def print_header(title: str, width: int = 40) -> None:
    """Print a formatted header. Side-effect only."""
    print("=" * width)
    print(title.center(width))
    print("=" * width)


def update_dict_inplace(d: dict[str, int], key: str, value: int) -> None:
    """Mutate a dictionary in place. The -> None signals mutation."""
    d[key] = value


logger = Logger()
logger.log("Application started")
logger.log("Processing data")
print(f"Messages: {logger.get_messages()}")

# mypy would flag this as an error:
# result: str = logger.log("test")  # error: incompatible types

print_header("Type Hints Demo")

data: dict[str, int] = {"a": 1}
update_dict_inplace(data, "b", 2)
print(f"Updated dict: {data}")

## Type Aliases with TypeAlias

Complex type annotations can become unwieldy. **Type aliases** give meaningful names
to complex types, improving readability. Python 3.10 introduced the `TypeAlias`
annotation (PEP 613) to make aliases explicit, and Python 3.12 added the `type`
statement (PEP 695) for an even cleaner syntax.

In [ ]:
from typing import TypeAlias

# Without aliases - hard to read
# def process(data: list[dict[str, list[tuple[int, float]]]]) -> dict[str, float]: ...

# With TypeAlias - explicit and clear
Coordinate: TypeAlias = tuple[float, float]
Matrix: TypeAlias = list[list[float]]
Headers: TypeAlias = dict[str, str]
UserId: TypeAlias = int
JsonValue: TypeAlias = str | int | float | bool | None | list["JsonValue"] | dict[str, "JsonValue"]


def distance(a: Coordinate, b: Coordinate) -> float:
    """Calculate Euclidean distance between two coordinates."""
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5


def transpose(m: Matrix) -> Matrix:
    """Transpose a matrix."""
    return [list(row) for row in zip(*m)]


def get_user_name(user_id: UserId) -> str:
    """Look up user by their ID."""
    users: dict[UserId, str] = {1: "Alice", 2: "Bob"}
    return users.get(user_id, "Unknown")


# Using the aliases
origin: Coordinate = (0.0, 0.0)
target: Coordinate = (3.0, 4.0)
print(f"Distance: {distance(origin, target)}")

m: Matrix = [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]
print(f"Original: {m}")
print(f"Transposed: {transpose(m)}")

print(f"User 1: {get_user_name(1)}")
print(f"User 99: {get_user_name(99)}")


# Python 3.12+ syntax (PEP 695) - even cleaner
# type Coordinate = tuple[float, float]
# type Matrix = list[list[float]]
# type JsonValue = str | int | float | bool | None | list[JsonValue] | dict[str, JsonValue]

## Callable Types

Functions are first-class objects in Python. The `Callable` type from `collections.abc`
(or `typing`) describes the signature of a callable: `Callable[[ParamTypes], ReturnType]`.

In [ ]:
from collections.abc import Callable


# A function that takes a transformation function as a parameter
def apply_to_all(
    items: list[str],
    transform: Callable[[str], str],
) -> list[str]:
    """Apply a transformation function to every item."""
    return [transform(item) for item in items]


# A function that returns a function (factory)
def make_multiplier(factor: float) -> Callable[[float], float]:
    """Create a multiplier function."""
    def multiplier(x: float) -> float:
        return x * factor
    return multiplier


# A callback type alias for clarity
ErrorHandler: TypeAlias = Callable[[Exception], None]


def safe_divide(
    a: float,
    b: float,
    on_error: ErrorHandler | None = None,
) -> float | None:
    """Divide a by b, with optional error handler callback."""
    try:
        return a / b
    except ZeroDivisionError as e:
        if on_error:
            on_error(e)
        return None


# Using apply_to_all
words = ["hello", "world", "python"]
print(f"Upper: {apply_to_all(words, str.upper)}")
print(f"Title: {apply_to_all(words, str.title)}")

# Using make_multiplier
double = make_multiplier(2.0)
triple = make_multiplier(3.0)
print(f"Double 5: {double(5.0)}")
print(f"Triple 5: {triple(5.0)}")

# Using safe_divide with error handler
def log_error(e: Exception) -> None:
    print(f"  Error caught: {e}")


print(f"10 / 3 = {safe_divide(10, 3)}")
print(f"10 / 0 = {safe_divide(10, 0, on_error=log_error)}")

## Putting It All Together

Here is a small, fully-annotated module that combines variable annotations, function
signatures, class hints, generics, Optional, and type aliases into a cohesive example.

In [ ]:
from typing import TypeAlias
from collections.abc import Callable

# Type aliases
Score: TypeAlias = float
StudentName: TypeAlias = str
GradeBook: TypeAlias = dict[StudentName, list[Score]]
GradingPolicy: TypeAlias = Callable[[list[Score]], str]


class Classroom:
    """A fully type-annotated classroom manager."""

    name: str
    _grades: GradeBook

    def __init__(self, name: str) -> None:
        self.name = name
        self._grades: GradeBook = {}

    def add_student(self, student: StudentName) -> None:
        """Register a student with an empty score list."""
        if student not in self._grades:
            self._grades[student] = []

    def record_score(self, student: StudentName, score: Score) -> None:
        """Record a score for an existing student."""
        if student not in self._grades:
            raise KeyError(f"Student {student!r} not found")
        self._grades[student].append(score)

    def get_average(self, student: StudentName) -> Score | None:
        """Return the student's average score, or None if no scores."""
        scores = self._grades.get(student, [])
        if not scores:
            return None
        return sum(scores) / len(scores)

    def apply_policy(
        self,
        student: StudentName,
        policy: GradingPolicy,
    ) -> str | None:
        """Apply a grading policy to a student's scores."""
        scores = self._grades.get(student)
        if scores is None:
            return None
        return policy(scores)

    def summary(self) -> dict[StudentName, Score | None]:
        """Return a summary of all students and their averages."""
        return {name: self.get_average(name) for name in self._grades}


# A grading policy as a typed callable
def letter_grade(scores: list[Score]) -> str:
    """Convert a list of scores to a letter grade."""
    avg = sum(scores) / len(scores) if scores else 0
    if avg >= 90:
        return "A"
    elif avg >= 80:
        return "B"
    elif avg >= 70:
        return "C"
    elif avg >= 60:
        return "D"
    return "F"


# Usage
room = Classroom("Python 101")
room.add_student("Alice")
room.add_student("Bob")

for score in [92.0, 88.5, 95.0]:
    room.record_score("Alice", score)
for score in [75.0, 82.0, 68.5]:
    room.record_score("Bob", score)

print(f"Classroom: {room.name}")
print(f"Averages: {room.summary()}")
print(f"Alice's grade: {room.apply_policy('Alice', letter_grade)}")
print(f"Bob's grade: {room.apply_policy('Bob', letter_grade)}")

## Summary

### Type Annotation Syntax
- **Variables**: `name: str = "Alice"` - annotate with `: Type` after the name
- **Function parameters**: `def greet(name: str)` - annotate each parameter
- **Return types**: `def greet(name: str) -> str` - use `->` after the parameter list
- **None return**: `def log(msg: str) -> None` - explicit for side-effect functions

### Built-in Generics (Python 3.9+)
- `list[int]`, `dict[str, float]`, `set[str]`, `tuple[int, str]`, `tuple[int, ...]`
- Nested generics: `dict[str, list[int]]`, `list[tuple[str, float]]`

### Union and Optional
- `Union[X, Y]` or `X | Y` (3.10+): value is one of multiple types
- `Optional[X]` or `X | None`: value can be `X` or `None`
- Prefer the `X | Y` syntax in modern Python

### Type Aliases
- `TypeAlias` (3.10+): `Coordinate: TypeAlias = tuple[float, float]`
- `type` statement (3.12+): `type Coordinate = tuple[float, float]`
- Use aliases to name complex types for readability

### Inspecting Annotations
- `typing.get_type_hints(obj)` is preferred over `obj.__annotations__`
- It resolves forward references and handles `from __future__ import annotations`